In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier
from tqdm import tqdm
from time import sleep

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential, callbacks, optimizers, utils, models, applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-08-12 16:57:14.684632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 16:57:15.384593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-12 16:57:15.384626: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-12 16:57:15.467500: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-12 16:57:17.335401: W tensorflow/stream_executor/platform/de

In [4]:
# folder_path = '/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/'
# dataframes = []
# for file_name in os.listdir(folder_path):
#     if file_name.endswith('.csv'):
#         file_path = os.path.join(folder_path, file_name)
#         df = pd.read_csv(file_path)
#         dataframes.append(df)
# merged_df = pd.concat(dataframes, ignore_index=True)

In [5]:
merged_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/long_sypnosis.csv")

merged_df.shape

(4624, 6)

In [6]:
merged_df.head(5)

,Unnamed: 0,movie,imdb_id,genre,image_url,sypnosis
0,0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [7]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [8]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(4624, 5)

In [9]:
merged_df.head(5)

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [10]:
merged_df["sypnosis"].notna().sum()

4624

In [11]:
merged_df

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."
...,...,...,...,...,...
4619,Bad Girls,tt0109198,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0109198&h=600&apik...,Four prostitutes join together to travel the O...
4620,Renegade,tt0276830,"['Adventure', ' Fantasy', ' Western']",http://img.omdbapi.com/?i=tt0276830&h=600&apik...,"In 1870, after a brutal run-in with an outlaw ..."
4621,Wild Wild West,tt0120891,"['Action', ' Comedy', ' Sci-Fi']",http://img.omdbapi.com/?i=tt0120891&h=600&apik...,The two best special agents in the Wild West m...
4622,The Ridiculous 6,tt2479478,"['Action', ' Adventure', ' Comedy']",http://img.omdbapi.com/?i=tt2479478&h=600&apik...,An outlaw who was raised by Native Americans d...


In [13]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

In [14]:
def cleaning(sentence):
    if isinstance(sentence, str):
    # Basic cleaning
        sentence = sentence.strip() ## remove whitespaces
        sentence = sentence.lower() ## lowercase 
        sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

        # Advanced cleaning
        for punctuation in string.punctuation:
            sentence = sentence.replace(punctuation, '') ## remove punctuation

        tokenized_sentence = word_tokenize(sentence) ## tokenize 
        stop_words = set(stopwords.words('english')) ## define stopwords

        tokenized_sentence_cleaned = [ ## remove stopwords
            w for w in tokenized_sentence if not w in stop_words
        ]

        lemmatized = [
            WordNetLemmatizer().lemmatize(word, pos = "v") 
            for word in tokenized_sentence_cleaned
        ]

        cleaned_sentence = ' '.join(word for word in lemmatized)

        return cleaned_sentence

In [15]:
# merged_df["clean_plot"] = merged_df["plot"].apply(cleaning)

In [16]:
merged_df["clean_sypnosis"] = merged_df["sypnosis"].apply(cleaning)

In [17]:
merged_df["clean_title"] = merged_df["movie"].apply(cleaning)

In [19]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [20]:
merged_df.shape

(4624, 7)

In [21]:
merged_df = merged_df.dropna()

In [22]:
merged_df.shape

(4624, 7)

In [23]:
#check the genre we have
merged_df["genre"].explode().value_counts()

Drama        2824
Comedy       1495
Adventure    1045
Action        994
Crime         806
Romance       647
Biography     591
Thriller      544
Horror        511
Mystery       504
Animation     405
Fantasy       387
Sci-Fi        360
Family        312
History       298
Music         261
War           201
Sport         187
Musical       106
Western       104
Film-Noir      59
Name: genre, dtype: int64

In [25]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df["genre"])

# transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])

In [26]:
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.shape

(4624, 28)

In [27]:
merged_df.head(5)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...,anime adaptation hindu epic ramayana lord ram ...,ramayana legend prince rama,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...,menace know joker wreak havoc chaos people got...,dark knight,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,...",miles morales catapult across multiverse encou...,spiderman across spiderverse,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive...",story prophet muhammad delivery message god allah,message,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.datasets import make_classification
# from collections import Counter

# # Generate a sample imbalanced dataset
# X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0, n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

# # Count the class distribution before undersampling
# print("Class distribution before undersampling:", Counter(y))

# # Apply random undersampling
# rus = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X, y)

# # Count the class distribution after undersampling
# print("Class distribution after undersampling:", Counter(y_resampled))

In [29]:
# X = merged_df[["clean_sypnosis","clean_title"]]
# y = merged_df.drop(columns=["movie","imdb_id","genre","image_url","sypnosis","clean_sypnosis","clean_title"])

In [252]:
condition1 =  (merged_df['Animation'] == 1) | (merged_df['Music'] == 1)  | (merged_df['Biography'] == 1) | (merged_df['Sport'] == 1) |(merged_df['Musical'] == 1) |(merged_df['War'] == 1)|(merged_df['Western'] == 1)|(merged_df['History'] == 1)|(merged_df['Film-Noir'] == 1)|(merged_df['Family'] == 1)

rows_to_drop1 = merged_df[condition1].index
test_df = merged_df.drop(rows_to_drop1)

test_df["genre"].explode().value_counts()

Drama        1446
Comedy        744
Action        703
Crime         608
Adventure     493
Horror        490
Thriller      480
Mystery       475
Romance       418
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [293]:
condition2 = (test_df['Comedy'] == 1)  & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)
rows_to_drop2 = test_df[condition2].index
test_df2 = test_df.drop(rows_to_drop2)

   
test_df2["genre"].explode().value_counts()

Drama        1267
Action        586
Crime         497
Horror        490
Thriller      480
Mystery       475
Romance       418
Comedy        383
Adventure     359
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [296]:
condition3 = (test_df2['Action'] == 1)  & (test_df2['Crime'] == 1)  & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)
# & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
rows_to_drop3 = test_df2[condition3].index
test_df3= test_df2.drop(rows_to_drop3)

   
test_df3["genre"].explode().value_counts()

/tmp/ipykernel_1086/3571784835.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows_to_drop3 = test_df2[condition3].index


Drama        1174
Horror        490
Action        485
Thriller      480
Mystery       475
Romance       418
Crime         396
Comedy        383
Adventure     351
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [297]:
test_df3.columns

Index(['movie', 'imdb_id', 'genre', 'image_url', 'sypnosis', 'clean_sypnosis',
       'clean_title', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [337]:
no_drama_df = test_df3.drop(columns=["Drama", "Biography", "Family","Film-Noir","History","Music","Musical","Sport", "War", "Western", "Animation"])

In [338]:
no_drama_df.columns

Index(['movie', 'imdb_id', 'genre', 'image_url', 'sypnosis', 'clean_sypnosis',
       'clean_title', 'Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy',
       'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller'],
      dtype='object')

In [344]:
condition_null = (no_drama_df['Action'] == 1) | (no_drama_df['Adventure'] == 1) | (no_drama_df['Comedy'] == 1)| (no_drama_df['Crime'] == 1)| (no_drama_df['Fantasy'] == 1)| (no_drama_df['Horror'] == 1)| (no_drama_df['Mystery'] == 1)| (no_drama_df['Romance'] == 1)| (no_drama_df['Sci-Fi'] == 1)| (no_drama_df['Thriller'] == 1)
no_drama_df1 =no_drama_df.loc[no_drama_df[condition_null].index]

In [346]:
no_drama_df1

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,0,0,0,0,0,0,0
5,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,0,0,0,0,0,1,0
6,The Lord of the Rings: The Fellowship of the Ring,tt0120737,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0120737&h=600&apik...,A meek Hobbit from the Shire and eight compani...,meek hobbit shire eight companion set journey ...,lord ring fellowship ring,1,1,0,0,0,0,0,0,0,0
7,The Lord of the Rings: The Two Towers,tt0167261,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167261&h=600&apik...,While Frodo and Sam edge closer to Mordor with...,frodo sam edge closer mordor help shifty gollu...,lord ring two tower,1,1,0,0,0,0,0,0,0,0
8,Kill Bill: The Whole Bloody Affair,tt6019206,"[Action, Crime, Thriller]",http://img.omdbapi.com/?i=tt6019206&h=600&apik...,The Bride must kill her ex-boss and lover Bill...,bride must kill exboss lover bill betray wed r...,kill bill whole bloody affair,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,Chaos Walking,tt2076822,"[Action, Adventure, Fantasy]",http://img.omdbapi.com/?i=tt2076822&h=600&apik...,Two unlikely companions embark on a perilous a...,two unlikely companion embark perilous adventu...,chaos walk,1,1,0,0,1,0,0,0,0,0
4614,The Dark Tower,tt1648190,"[Action, Adventure, Fantasy]",http://img.omdbapi.com/?i=tt1648190&h=600&apik...,A boy haunted by visions of a dark tower from ...,boy haunt visions dark tower parallel reality ...,dark tower,1,1,0,0,1,0,0,0,0,0
4619,Bad Girls,tt0109198,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0109198&h=600&apik...,Four prostitutes join together to travel the O...,four prostitute join together travel old west ...,bad girls,1,1,0,0,0,0,0,0,0,0
4621,Wild Wild West,tt0120891,"[Action, Comedy, Sci-Fi]",http://img.omdbapi.com/?i=tt0120891&h=600&apik...,The two best special agents in the Wild West m...,two best special agents wild west must save pr...,wild wild west,1,0,1,0,0,0,0,0,1,0


In [347]:
no_drama_df1.reset_index(inplace=True)

In [348]:
no_drama_df1 = no_drama_df1.drop(columns=["index"])

In [349]:
no_drama_df1.head(3)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,0,0,0,0,0,0,0
1,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,0,0,0,0,0,1,0
2,The Lord of the Rings: The Fellowship of the Ring,tt0120737,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0120737&h=600&apik...,A meek Hobbit from the Shire and eight compani...,meek hobbit shire eight companion set journey ...,lord ring fellowship ring,1,1,0,0,0,0,0,0,0,0


In [382]:
no_drama_df1.to_csv("filtered.csv")

In [383]:
no_drama_df1["genre"].explode().value_counts()

Drama        1112
Horror        490
Action        485
Thriller      480
Mystery       475
Romance       418
Crime         396
Comedy        383
Adventure     351
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [41]:
# condition = (merged_df['Drama'] == 1) & (merged_df['Comedy'] == 1)
# rows_to_drop = merged_df[condition].head(500).index
# balancing_df = merged_df.drop(rows_to_drop)
# balancing_df.shape
# balancing_df["genre"].explode().value_counts()

In [42]:
# condition1 = (merged_df['Drama'] == 1)&(merged_df['Romance'] != 1)& (merged_df['Sci-Fi'] != 1)& (merged_df['Fantasy'] != 1)
# # & (merged_df['Crime'] != 1)& (merged_df['Romance'] != 1)& (merged_df['Thriller'] != 1)& (merged_df['Biography'] != 1)& (merged_df['Horror'] != 1)& (merged_df['Mystery'] != 1)& (merged_df['Animation'] != 1)& (merged_df['Sci-Fi'] != 1)& (merged_df['Fantasy'] != 1)& (merged_df['History'] != 1)& (merged_df['Family'] != 1)& (merged_df['Music'] != 1)
# # & (merged_df['War'] == 1)& (merged_df['Sport'] == 1)& (merged_df['Western'] == 1)& (merged_df['Musical'] == 1)& (merged_df['Film-Noir'] == 1)
# # & (merged_df['Adventure'] == 1)& (merged_df['Action'] != 1)
# # & (merged_df['Comedy'] != 1)

# rows_to_drop1 = merged_df[condition1].index
# # .head(1000).index
# test_df = merged_df.drop(rows_to_drop1)


   
# test_df["genre"].explode().value_counts()

In [43]:
# condition2 = (test_df['Comedy'] == 1)  & (test_df['Adventure'] == 1)
# # & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
# rows_to_drop2 = test_df[condition2].index
# test_df2 = test_df.drop(rows_to_drop2)

   
# test_df2["genre"].explode().value_counts()

In [44]:
# condition3 = (test_df['Music'] == 1) | (test_df['Biography'] == 1) | (test_df['Sport'] == 1) |(test_df['Musical'] == 1) |(test_df['War'] == 1)|(test_df['Western'] == 1)|(test_df['History'] == 1)|(test_df['Film-Noir'] == 1)|(test_df['Family'] == 1)

# rows_to_drop3 = test_df[condition3].index
# test_df3 = test_df.drop(rows_to_drop3)

   
# test_df3["genre"].explode().value_counts()

In [45]:
# condition2 = (balancing_df['Drama'] == 1)
# rows_to_drop = balancing_df[condition2].head(500).index
# balancing_df2 = balancing_df.drop(rows_to_drop)
# balancing_df2["genre"].explode().value_counts()

In [350]:
no_drama_df1.head(5)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,0,0,0,0,0,0,0
1,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,0,0,0,0,0,1,0
2,The Lord of the Rings: The Fellowship of the Ring,tt0120737,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0120737&h=600&apik...,A meek Hobbit from the Shire and eight compani...,meek hobbit shire eight companion set journey ...,lord ring fellowship ring,1,1,0,0,0,0,0,0,0,0
3,The Lord of the Rings: The Two Towers,tt0167261,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167261&h=600&apik...,While Frodo and Sam edge closer to Mordor with...,frodo sam edge closer mordor help shifty gollu...,lord ring two tower,1,1,0,0,0,0,0,0,0,0
4,Kill Bill: The Whole Bloody Affair,tt6019206,"[Action, Crime, Thriller]",http://img.omdbapi.com/?i=tt6019206&h=600&apik...,The Bride must kill her ex-boss and lover Bill...,bride must kill exboss lover bill betray wed r...,kill bill whole bloody affair,1,0,0,1,0,0,0,0,0,1


## Split Data and Vectorize

In [351]:
X = no_drama_df1[["movie","clean_sypnosis"]]
y = no_drama_df1.drop(columns=["movie","imdb_id","genre","image_url","sypnosis","clean_sypnosis","clean_title"])

In [352]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [353]:
X_train.shape

(1347, 2)

In [354]:
X_train.head(5)

,movie,clean_sypnosis
485,Manhattan Murder Mystery,middleaged couple suspect foul play neighbor w...
660,Targets,age horror star question place modern hollywoo...
1266,Unsane,young woman involuntarily commit mental instit...
1785,Self/less,die real estate mogul transfer consciousness h...
975,The Serpent and the Rainbow,anthropologist go haiti hear rumor drug use bl...


In [355]:
X_test.head(5)

,movie,clean_sypnosis
156,Dawn of the Dead,nurse policeman young marry couple salesman su...
64,Captain America: The Winter Soldier,steve rogers struggle embrace role modern worl...
876,Bruce Almighty,whiny news reporter give chance step gods shoe...
1779,War of the Worlds,alien invasion threaten future humanity catast...
1091,The Curse of Frankenstein,await execution murder baron victor frankenste...


In [356]:
y_train.shape 

(1347, 10)

In [357]:
y_train.head(5)

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
485,0,0,1,0,0,0,1,0,0,0
660,0,0,0,1,0,0,0,0,0,1
1266,0,0,0,0,0,1,1,0,0,0
1785,1,0,0,0,0,0,1,0,1,0
975,0,0,0,0,1,1,0,0,0,0


In [358]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=0.15, max_features=10000)
# tf_idf_title_vectorizer = TfidfVectorizer(min_df=10, max_features=10000)
# tf_idf_vectorizer = TfidfVectorizer()

# Training it on the texts
X_train_vec = pd.DataFrame(tf_idf_vectorizer.fit_transform(X_train["clean_sypnosis"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
# X_train_title_vec = pd.DataFrame(tf_idf_title_vectorizer.fit_transform(X_train["clean_title"]).toarray(),
#                  columns = tf_idf_title_vectorizer.get_feature_names_out())
X_test_vec =  pd.DataFrame(tf_idf_vectorizer.transform(X_test["clean_sypnosis"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
# X_test_title_vec =  pd.DataFrame(tf_idf_title_vectorizer.transform(X_test["clean_title"]).toarray(),
#                  columns = tf_idf_title_vectorizer.get_feature_names_out())


In [55]:
# X_test_title_vec

In [56]:
# X_train_vec = pd.concat([X_train_plot_vec,X_train_title_vec], axis=1)

In [57]:
# X_train_vec

In [58]:
# X_train_vec.shape

In [59]:
# X_test_vec = pd.concat([X_test_plot_vec,X_test_title_vec], axis=1)

In [359]:
X_train_vec.head(5)

,abandon,able,accept,accidentally,across,act,action,actually,admit,age,...,world,would,wound,write,wrong,year,years,yet,york,young
0,0.000000,0.000000,0.0,0.000000,0.000000,0.022383,0.0,0.098092,0.000000,0.027498,...,0.000000,0.034747,0.000000,0.000000,0.0,0.00000,0.034557,0.0,0.000000,0.030715
1,0.028172,0.000000,0.0,0.030224,0.000000,0.024439,0.0,0.000000,0.000000,0.060050,...,0.019989,0.000000,0.055772,0.026349,0.0,0.00000,0.000000,0.0,0.030479,0.067073
2,0.000000,0.000000,0.0,0.000000,0.037304,0.078404,0.0,0.000000,0.000000,0.000000,...,0.000000,0.060858,0.000000,0.000000,0.0,0.00000,0.030263,0.0,0.000000,0.026898
3,0.030696,0.023801,0.0,0.065865,0.000000,0.000000,0.0,0.029175,0.029136,0.000000,...,0.021780,0.020669,0.000000,0.000000,0.0,0.05609,0.020556,0.0,0.066420,0.493310
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000


In [62]:
# X_test_vec.shape

In [360]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

## OneVsRest logistic regression

### Simple OneVsRest logistic regression

In [361]:
log = LogisticRegression()
model_test_1 = OneVsRestClassifier(log)

model_test_1.fit(X_train_vec, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [362]:
y_pred = model_test_1.predict(X_test_vec)

In [317]:
accuracy_score(y_test, y_pred)

0.12897822445561138

In [363]:
f1_score(y_test, y_pred, average="micro")

0.4574095682613769

In [364]:
results = pd.DataFrame(y_pred, columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0


In [69]:
# multilabel_binarizer.inverse_transform(y_pred)[1]

In [365]:
y_test[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
156,1,0,0,0,0,1,0,0,0,0
64,1,1,0,0,0,0,0,0,1,0
876,0,0,1,0,1,0,0,0,0,0
1779,1,1,0,0,0,0,0,0,1,0
1091,0,0,0,0,0,1,0,0,1,1


#### Grouping the topic

In [168]:
# from sklearn.decomposition import LatentDirichletAllocation

# # Instantiate the LDA 
# n_components = 7
# lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 500)

# # Fit the LDA on the vectorized documents
# lda_model.fit(X_train_vec)

LatentDirichletAllocation(max_iter=500, n_components=7)

In [169]:
# X_train_topic_mixture = lda_model.transform(X_train_vec)
# X_train_topic_df = pd.DataFrame(X_train_topic_mixture)
# X_train_topic_df

,0,1,2,3,4,5,6
0,0.011503,0.011508,0.011504,0.930973,0.011503,0.011504,0.011504
1,0.008391,0.008392,0.008391,0.949652,0.008391,0.008391,0.008391
2,0.011082,0.011085,0.011083,0.933502,0.011082,0.011083,0.011083
3,0.011441,0.011441,0.011441,0.931354,0.011441,0.011441,0.011442
4,0.011458,0.011459,0.011459,0.931249,0.011458,0.011458,0.011458
...,...,...,...,...,...,...,...
1499,0.012332,0.012334,0.012332,0.926006,0.012332,0.012333,0.012332
1500,0.020110,0.020111,0.020110,0.879335,0.020110,0.020113,0.020110
1501,0.011785,0.011785,0.011785,0.929290,0.011785,0.011785,0.011785
1502,0.029440,0.029407,0.029403,0.823541,0.029403,0.029403,0.029403


In [170]:
# X_test_topic_mixture = lda_model.transform(X_test_vec)
# X_test_topic_df = pd.DataFrame(X_test_topic_mixture)
# X_test_topic_df

,0,1,2,3,4,5,6
0,0.011115,0.011115,0.011115,0.933312,0.011114,0.011115,0.011115
1,0.011312,0.011312,0.011312,0.932129,0.011311,0.011312,0.011313
2,0.017869,0.017871,0.017869,0.892785,0.017869,0.017869,0.017869
3,0.011092,0.011093,0.011093,0.933445,0.011092,0.011093,0.011093
4,0.022049,0.022049,0.022049,0.867705,0.022049,0.022049,0.022050
...,...,...,...,...,...,...,...
640,0.019668,0.019669,0.019670,0.881988,0.019668,0.019670,0.019668
641,0.012648,0.012650,0.012649,0.924108,0.012648,0.012648,0.012648
642,0.011069,0.011069,0.011069,0.933585,0.011069,0.011069,0.011069
643,0.016895,0.016894,0.016895,0.898632,0.016894,0.016894,0.016895


### Training OneVsRest using topic grouping as input

In [171]:
# model_test_2 = OneVsRestClassifier(log)
# model_test_2.fit(X_train_topic_df, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [172]:
# y_pred_2 = model_test_2.predict(X_test_topic_df)
# accuracy_score(y_test, y_pred_2)

0.0

In [173]:
# results = pd.DataFrame(y_pred_2, columns=["Action","Adventure","Animation","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
# results[:5]

,Action,Adventure,Animation,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0


In [174]:
# y_test[:5]

,Action,Adventure,Animation,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
1301,0,0,0,0,0,0,1,0,0,0,0
782,0,0,0,1,0,1,0,0,0,0,0
1876,0,0,0,0,0,0,0,0,1,0,0
784,0,0,0,1,0,0,0,0,0,0,0
1786,0,0,0,0,1,0,0,1,0,0,0


### Using both topic grouping as input + OneVsRest 

In [ ]:
# model_test_3 = OneVsRestClassifier(log)
# model_test_3.fit(X_train_topic_df, y_train)
# model_test_3.fit(X_train_vec, y_train)

In [ ]:
# y_pred_3 = model_test_2.predict(X_test_topic_df)
# accuracy_score(y_test, y_pred_3)

## SGD

In [366]:
sgd = SGDClassifier(random_state = 42)
sgd_model = OneVsRestClassifier(sgd)
sgd_model.fit(X_train_vec,y_train)

OneVsRestClassifier(estimator=SGDClassifier(random_state=42))

In [367]:
y_pred_sgd1 = sgd_model.predict(X_test_vec)

In [ ]:
# multilabel_binarizer.inverse_transform(y_pred_sgd1)[0:10]

In [368]:
accuracy_score(y_test, y_pred_sgd1)

0.14878892733564014

In [369]:
f1_score(y_test, y_pred_sgd1, average="micro")

0.5757575757575758

In [370]:
results = pd.DataFrame(y_pred_sgd1, columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[0:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0


In [371]:
y_test[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
156,1,0,0,0,0,1,0,0,0,0
64,1,1,0,0,0,0,0,0,1,0
876,0,0,1,0,1,0,0,0,0,0
1779,1,1,0,0,0,0,0,0,1,0
1091,0,0,0,0,0,1,0,0,1,1


In [120]:
# condition = no_drama_df['clean_sypnosis'].isin(X_test['clean_sypnosis'][:5])
# filtered_df = no_drama_df[condition]
# filtered_df["genre"]

782           [Comedy, Fantasy]
784                    [Comedy]
1301                   [Horror]
1786    [Crime, Drama, Mystery]
1876           [Drama, Romance]
Name: genre, dtype: object

In [ ]:
# multilabel_binarizer.inverse_transform(y_pred_sgd1)[2]

In [ ]:
# test_df3[test_df3.clean_sypnosis == X_test.clean_sypnosis.iloc[2]]

## Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(random_state = 42)
rfc_model = OneVsRestClassifier(rfc)

rfc_model.fit(X_train_vec,y_train)

In [ ]:
y_pred_rfc1 = rfc_model.predict(X_test_vec)

In [ ]:
results = pd.DataFrame(y_pred_rfc1, columns=["Action","Adventure","Animation","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[4:5]

In [ ]:
no_drama_df[no_drama_df.clean_sypnosis == X_test.clean_sypnosis.iloc[4]]

In [ ]:
accuracy_score(y_test, y_pred_rfc1)

In [ ]:
f1_score(y_test, y_pred_rfc1, average="micro")

## Poly SVC

In [ ]:
# X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_sypnosis'], y, test_size = 0.3, random_state = 0)
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
# X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
# sgd = SGDClassifier()
# lr = LogisticRegression(C=1.0)
# svc = SVC(kernel='poly')
# nb = MultinomialNB()
# # One vs Restclassifier
# model = OneVsRestClassifier(svc)
# model.fit(X_train_tfidf1, y_train1)
# y_pred1 = model.predict(X_val_tfidf1)
# print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')
# print(f'F1 score for SVC poly model is {f1_score(y_val1, y_pred1, average="micro")}')

In [372]:
# sgd = SGDClassifier()
# lr = LogisticRegression(C=1.0)
# svc = SVC(kernel='poly')
# nb = MultinomialNB()
# # One vs Restclassifier
# model = OneVsRestClassifier(svc)
# model.fit(X_train_vec, y_train)
# y_pred_sgd2 = model.predict(X_test_vec)
# print(f'Accuracy score for SVC poly model is {accuracy_score(y_test, y_pred_sgd2)}')
# print(f'F1 score for SVC poly model is {f1_score(y_test, y_pred_sgd2, average="micro")}')

Accuracy score for SVC poly model is 0.1245674740484429
F1 score for SVC poly model is 0.4657534246575343


In [ ]:
# X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_sypnosis'], y, test_size = 0.3, random_state = 0)
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
# X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
# sgd = SGDClassifier()
# lr = LogisticRegression(C=1.0)
# svc = SVC(kernel='poly')
# nb = MultinomialNB()
# # One vs Restclassifier
# model = OneVsRestClassifier(svc)
# model.fit(X_train_tfidf1, y_train1)
# y_pred1 = model.predict(X_val_tfidf1)
# print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')
# print(f'F1 score for SVC poly model is {f1_score(y_val1, y_pred1)}')

## Label Powerset

In [373]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(X_train_vec, y_train)
# predict
predictions_lp = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_lp))
print("F1 = ",f1_score(y_test,predictions_lp, average="micro"))
print("\n")

Accuracy =  0.22837370242214533
F1 =  0.5422446406052963




/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [374]:
predictions_lp.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [375]:
results = pd.DataFrame(predictions_lp.toarray(), columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,1,0,0
3,0,0,0,0,0,1,1,0,0,0
4,0,0,0,1,0,0,1,0,0,0


In [376]:
y_test[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
156,1,0,0,0,0,1,0,0,0,0
64,1,1,0,0,0,0,0,0,1,0
876,0,0,1,0,1,0,0,0,0,0
1779,1,1,0,0,0,0,0,0,1,0
1091,0,0,0,0,0,1,0,0,1,1


In [89]:
# no_drama_df[no_drama_df.clean_sypnosis == X_test.clean_sypnosis.iloc[4]]

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
396,In July,tt0177858,"[Adventure, Comedy, Romance]",http://img.omdbapi.com/?i=tt0177858&h=600&apik...,"A young, insecure teacher embarks on a journey...",young insecure teacher embark journey europe t...,july,0,1,0,1,0,0,0,0,1,0,0


In [ ]:
# multilabel_binarizer.inverse_transform(predictions_lp)[0:10]

In [ ]:
# condition = test_df3['clean_sypnosis'].isin(X_test['clean_sypnosis'][:10])
# filtered_df = test_df3[condition]
# filtered_df["genre"]

## Classifier Chains

In [377]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_vec, y_train)
# predict
y_pred_cc = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,y_pred_cc))
print("\n")

Accuracy =  0.21280276816608998




In [378]:
y_pred_cc.toarray()

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [380]:
results = pd.DataFrame(y_pred_cc.toarray(), columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[:3]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# multilabel_binarizer.inverse_transform(predictions)[0:3]

In [381]:
condition = no_drama_df['clean_sypnosis'].isin(X_test['clean_sypnosis'][:3])
filtered_df = no_drama_df[condition]
filtered_df["genre"]

102     [Action, Adventure, Sci-Fi]
294                [Action, Horror]
2765              [Comedy, Fantasy]
Name: genre, dtype: object

## BERT

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd